In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

In [2]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [3]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [4]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [5]:
# help(MTCNN)

In [44]:
import glob
img_list = glob.glob('./datasets/FGNET/images/*')
img_list.sort()
print(img_list)

['./datasets/FGNET/images/001A02.JPG', './datasets/FGNET/images/001A05.JPG', './datasets/FGNET/images/001A08.JPG', './datasets/FGNET/images/001A10.JPG', './datasets/FGNET/images/001A14.JPG', './datasets/FGNET/images/001A16.JPG', './datasets/FGNET/images/001A18.JPG', './datasets/FGNET/images/001A19.JPG', './datasets/FGNET/images/001A22.JPG', './datasets/FGNET/images/001A28.JPG', './datasets/FGNET/images/001A29.JPG', './datasets/FGNET/images/001A33.JPG', './datasets/FGNET/images/001A40.JPG', './datasets/FGNET/images/001A43a.JPG', './datasets/FGNET/images/001A43b.JPG', './datasets/FGNET/images/002A03.JPG', './datasets/FGNET/images/002A04.JPG', './datasets/FGNET/images/002A05.JPG', './datasets/FGNET/images/002A07.JPG', './datasets/FGNET/images/002A12.JPG', './datasets/FGNET/images/002A15.JPG', './datasets/FGNET/images/002A16.JPG', './datasets/FGNET/images/002A18.JPG', './datasets/FGNET/images/002A20.JPG', './datasets/FGNET/images/002A21.JPG', './datasets/FGNET/images/002A23.JPG', './datase

In [6]:
import glob
img_list = glob.glob('./datasets/FGNET/images/*')
img_list = sort(img_list)

from PIL import Image
from torchvision.transforms import transforms
from tqdm import tqdm
embeddings = []
names = []
for img in tqdm(img_list):
    img_mat = Image.open(img).convert('RGB')
#     img_mat = (transforms.ToTensor())(img_mat)
#     img_mat = img_mat.to(device)
    x_aligned, prob = mtcnn(img_mat, return_prob=True)
    if x_aligned is not None:
#         print('Face detected with probability: {:8f}'.format(prob))
        x_aligned = x_aligned.unsqueeze(0).to(device)
        embeddings.append(resnet(x_aligned).detach().cpu())
        names.append(img.split('/')[-1].split('.')[0])
    else:
        print(img)
#         print(names.append(img.split('/')[-1].split('.')[0]))

  5%|▌         | 51/1002 [00:02<00:49, 19.22it/s]

./datasets/FGNET/images/057A00.JPG


 54%|█████▍    | 541/1002 [00:34<00:28, 16.11it/s]

./datasets/FGNET/images/050A00.JPG


 72%|███████▏  | 725/1002 [00:45<00:14, 19.62it/s]

./datasets/FGNET/images/014A03.JPG


100%|██████████| 1002/1002 [01:00<00:00, 16.51it/s]


In [7]:
embeddings_mat = torch.stack(embeddings)
embeddings_mat = embeddings_mat.squeeze().numpy()

In [8]:
import scipy.io as sio
sio.savemat('face_embedding.mat', {'embeddings':embeddings_mat, 'names':names})

In [9]:
E = sio.loadmat('face_embedding.mat')

In [10]:
E['embeddings'].shape

(999, 512)

In [11]:
len(list(E['names']))

999

In [17]:
torch.FloatTensor(E['embeddings'][0]).size()

torch.Size([512])

In [18]:
len(list(E['names']))

999

In [34]:
# person_index
int(E['names'][0][0:3])

57

In [53]:
# age
int(E['names'][14][4:6])

torch.Size([3])

In [ ]:
for i, e in enumerate(embeddings):
    if len(e.shape)>2:
        print(i)

In [13]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('./datasets/FGNET_per_person')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

In [14]:
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Face detected with probability: 0.999298
Face detected with probability: 0.999659
Face detected with probability: 0.994806
Face detected with probability: 0.999999
Face detected with probability: 0.998275
Face detected with probability: 0.999739
Face detected with probability: 0.985511
Face detected with probability: 0.997530
Face detected with probability: 0.999390
Face detected with probability: 0.999995
Face detected with probability: 0.999997
Face detected with probability: 0.999576
Face detected with probability: 0.999994
Face detected with probability: 0.999977
Face detected with probability: 0.999935


In [15]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

In [16]:
names

['001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001',
 '001']

In [17]:
print(embeddings.size())

torch.Size([15, 512])


In [18]:
embeddings[0].numpy()

array([-5.13848513e-02,  6.01241402e-02,  1.37310950e-02, -3.80063355e-02,
       -3.27245868e-03,  3.95311452e-02,  3.05108856e-02, -1.10700671e-02,
       -7.98700936e-03,  1.57174040e-02,  1.43071428e-01, -6.48481166e-03,
        7.78135955e-02, -7.14392588e-03, -3.88496928e-03,  2.03087390e-03,
        2.27166452e-02,  4.73824814e-02, -2.02051047e-02, -2.54703742e-02,
        1.39899636e-02, -3.74177098e-02, -1.50832755e-03, -6.56636357e-02,
       -3.40746120e-02, -6.50594160e-02, -4.65669222e-02, -3.04528177e-02,
       -3.84333655e-02,  5.18139377e-02,  2.98517942e-02,  2.51667388e-02,
        5.53532615e-02,  2.17218883e-02,  4.16480266e-02,  5.16047552e-02,
        3.71043161e-02,  2.20369454e-02,  2.67500691e-02,  2.89999954e-02,
       -3.33624259e-02, -7.06516504e-02,  2.10866798e-02,  2.54906826e-02,
       -2.78093275e-02,  2.49276799e-03, -2.62248470e-03, -2.05740449e-03,
        1.04384087e-02, -4.39664163e-02, -3.13088186e-02, -5.61196208e-02,
       -1.47552872e-02,  

In [19]:
import csv

f = open('./datasets/FGNET_per_person/1.csv','w',encoding='utf-8')
csv_writer = csv.writer(f)

for i in range(embeddings.size()[0]):
    csv_writer.writerow(embeddings[i].numpy())
    
f.close()

In [20]:
im

NameError: name 'im' is not defined